In [1]:
newick_file = '../libsbn/data/hello_singleton.nwk'
fasta_file = '../libsbn/data/hello.fasta'

In [2]:
import substitution_model
import numpy as np
import tensorflow as tf

kappa = np.array(2.)
pi = np.array([0.27, 0.26, 0.24, 0.23])
eigendecomp = [x.numpy() for x in substitution_model.hky_eigendecomposition(pi, kappa)]
eigendecomp

[array([[ 1.        ,  1.96078432,  0.        ,  0.47058824],
        [ 1.        , -2.04081631,  0.46938777,  0.        ],
        [ 1.        ,  1.96078432,  0.        , -0.52941179],
        [ 1.        , -2.04081631, -0.53061223,  0.        ]]),
 array([ 0.        , -1.00180328, -1.49268687, -1.51272285]),
 array([[ 0.27  ,  0.26  ,  0.24  ,  0.23  ],
        [ 0.1323, -0.1326,  0.1176, -0.1173],
        [ 0.    ,  1.    ,  0.    , -1.    ],
        [ 1.    ,  0.    , -1.    ,  0.    ]])]

### Likelihood value

In [3]:
import tensorflow_scan_likelihood
from importlib import reload
reload(tensorflow_scan_likelihood)

tf_likelihood = tensorflow_scan_likelihood.TensorflowScanLikelihood(newick_file=newick_file, fasta_file=fasta_file)
branch_lengths = tf_likelihood.get_init_branch_lengths()
tf_likelihood.compute_likelihood(branch_lengths, pi, eigendecomp)

-84.5624981081271

### Full parameter gradients

In [4]:
with tf.GradientTape() as t:
    branch_lengths = tf.convert_to_tensor(branch_lengths)
    kappa = tf.convert_to_tensor(kappa)
    pi = tf.convert_to_tensor(pi)
    t.watch(branch_lengths)
    t.watch(kappa)
    t.watch(pi)
    eigendecomp = substitution_model.hky_eigendecomposition(pi, kappa)
    likelihood = tf_likelihood.compute_likelihood_tf(branch_lengths, pi, eigendecomp)
t.gradient(likelihood, [branch_lengths, kappa, pi])

Instructions for updating:
This op will be removed after the deprecation date. Please switch to tf.sets.difference().


[<tf.Tensor: id=859, shape=(4,), dtype=float64, numpy=array([ 21.13854016,  -5.43991438, -17.56636543, -17.56636543])>,
 <tf.Tensor: id=1190, shape=(), dtype=float64, numpy=-0.1310531416474613>,
 <tf.Tensor: id=1191, shape=(4,), dtype=float64, numpy=array([145.36954667, 139.26020098, 179.57984775, 128.4783711 ])>]

### Q gradient - HKY

In [138]:
with tf.GradientTape() as t:
    t.watch(kappa)
    t.watch(pi)
    hky_q = substitution_model.hky_q_matrix(pi, kappa)
t.jacobian(hky_q, [kappa, pi])

[<tf.Tensor: id=13500, shape=(4, 4), dtype=float64, numpy=
 array([[-0.24,  0.  ,  0.24,  0.  ],
        [ 0.  , -0.23,  0.  ,  0.23],
        [ 0.27,  0.  , -0.27,  0.  ],
        [ 0.  ,  0.26,  0.  , -0.26]])>,
 <tf.Tensor: id=13508, shape=(4, 4, 4), dtype=float64, numpy=
 array([[[ 0., -1., -2., -1.],
         [ 0.,  1.,  0.,  0.],
         [ 0.,  0.,  2.,  0.],
         [ 0.,  0.,  0.,  1.]],
 
        [[ 1.,  0.,  0.,  0.],
         [-1.,  0., -1., -2.],
         [ 0.,  0.,  1.,  0.],
         [ 0.,  0.,  0.,  2.]],
 
        [[ 2.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0.],
         [-2., -1.,  0., -1.],
         [ 0.,  0.,  0.,  1.]],
 
        [[ 1.,  0.,  0.,  0.],
         [ 0.,  2.,  0.,  0.],
         [ 0.,  0.,  1.,  0.],
         [-1., -2., -1.,  0.]]])>]

#### Kappa

In [144]:
reload(substitution_model)

hky_kappa_differential = substitution_model.hky_kappa_differential(pi)
hky_kappa_differential

<tf.Tensor: id=14776, shape=(4, 4), dtype=float64, numpy=
array([[-0.24,  0.  ,  0.24,  0.  ],
       [ 0.  , -0.23,  0.  ,  0.23],
       [ 0.27,  0.  , -0.27,  0.  ],
       [ 0.  ,  0.26,  0.  , -0.26]])>

#### Frequencies

### Normalisation gradient

In [145]:
with tf.GradientTape() as t:
    t.watch(kappa)
    t.watch(pi)
    hky_q = substitution_model.hky_q_matrix(pi, kappa)
    hky_norm_const = substitution_model.normalising_constant(hky_q, pi)
    hky_q_norm = hky_q / hky_norm_const
t.jacobian(hky_q_norm, [kappa])

[<tf.Tensor: id=15627, shape=(4, 4), dtype=float64, numpy=
 array([[ 0.00216378, -0.06502588,  0.120385  , -0.0575229 ],
        [-0.06752688,  0.01218182, -0.06002389,  0.11536895],
        [ 0.13543312, -0.06502588, -0.01288434, -0.0575229 ],
        [-0.06752688,  0.13041708, -0.06002389, -0.00286631]])>]

In [146]:
substitution_model.normalised_differential(hky_kappa_differential, hky_q_norm, hky_norm_const, pi)

<tf.Tensor: id=15635, shape=(4, 4), dtype=float64, numpy=
array([[ 0.00216378, -0.06502588,  0.120385  , -0.0575229 ],
       [-0.06752688,  0.01218182, -0.06002389,  0.11536895],
       [ 0.13543312, -0.06502588, -0.01288434, -0.0575229 ],
       [-0.06752688,  0.13041708, -0.06002389, -0.00286631]])>

### Transition probability gradient

In [170]:
with tf.GradientTape() as t:
    t.watch(kappa)
    t.watch(pi)
    eigendecomp = substitution_model.hky_eigendecomposition(pi, kappa)
    transition_probs_2 = substitution_model.transition_probs(eigendecomp, branch_lengths[:2])

transition_probs_inv = tf.linalg.inv(substitution_model.transition_probs(eigendecomp, branch_lengths[:2]))
tf.linalg.matmul(transition_probs_inv, t.jacobian(transition_probs_2, [kappa])[0])

<tf.Tensor: id=21279, shape=(2, 4, 4), dtype=float64, numpy=
array([[[ 0.00021638, -0.00650259,  0.0120385 , -0.00575229],
        [-0.00675269,  0.00121818, -0.00600239,  0.0115369 ],
        [ 0.01354331, -0.00650259, -0.00128843, -0.00575229],
        [-0.00675269,  0.01304171, -0.00600239, -0.00028663]],

       [[ 0.00021638, -0.00650259,  0.0120385 , -0.00575229],
        [-0.00675269,  0.00121818, -0.00600239,  0.0115369 ],
        [ 0.01354331, -0.00650259, -0.00128843, -0.00575229],
        [-0.00675269,  0.01304171, -0.00600239, -0.00028663]]])>

In [163]:
reload(substitution_model)

q_diff_norm = substitution_model.normalised_differential(hky_kappa_differential, hky_q_norm, hky_norm_const, pi)
substitution_model.transition_probs_differential(q_diff_norm, eigendecomp, branch_lengths[:2])

<tf.Tensor: id=16953, shape=(2, 4, 4), dtype=float64, numpy=
array([[[ 0.00021638, -0.00650259,  0.0120385 , -0.00575229],
        [-0.00675269,  0.00121818, -0.00600239,  0.0115369 ],
        [ 0.01354331, -0.00650259, -0.00128843, -0.00575229],
        [-0.00675269,  0.01304171, -0.00600239, -0.00028663]],

       [[ 0.00021638, -0.00650259,  0.0120385 , -0.00575229],
        [-0.00675269,  0.00121818, -0.00600239,  0.0115369 ],
        [ 0.01354331, -0.00650259, -0.00128843, -0.00575229],
        [-0.00675269,  0.01304171, -0.00600239, -0.00028663]]])>

### Sum-product gradient

In [181]:
import tensorflow_scan_likelihood
from importlib import reload
reload(tensorflow_scan_likelihood)

tf_scan_likelihood = tensorflow_scan_likelihood.TensorflowScanLikelihood(newick_file=newick_file, fasta_file=fasta_file)
tf_scan_likelihood.init_postorder_partials(fasta_file)
tf_scan_likelihood.compute_postorder_partials(transition_probs)
tf_scan_likelihood.init_preorder_partials(pi)
tf_scan_likelihood.compute_preorder_partials(transition_probs)

#### Branch length gradient

In [191]:
reload(substitution_model)

hky_q = substitution_model.normalise(substitution_model.hky_q_matrix(pi, kappa), pi)
hky_q

<tf.Tensor: id=23052, shape=(4, 4), dtype=float64, numpy=
array([[-0.97174915,  0.26046884,  0.48086556,  0.23041475],
       [ 0.27048688, -0.97174915,  0.24043278,  0.46082949],
       [ 0.54097375,  0.26046884, -1.03185734,  0.23041475],
       [ 0.27048688,  0.52093769,  0.24043278, -1.03185734]])>

In [174]:
tf_scan_likelihood.compute_edge_derivatives(tf.expand_dims(hky_q, 0))

<tf.Tensor: id=21533, shape=(4,), dtype=float64, numpy=array([ 18.75984318,  -4.54135731, -17.46598446, -17.37293861])>

#### Kappa gradient

In [182]:
transition_prob_differentials = substitution_model.transition_probs_differential(q_diff_norm, eigendecomp, branch_lengths)
tf.reduce_sum(tf_scan_likelihood.compute_edge_derivatives(transition_prob_differentials))

<tf.Tensor: id=22482, shape=(), dtype=float64, numpy=-0.1310531416474617>

#### Frequencies gradients

In [204]:
site_likelihoods = tf.reduce_sum(tf_scan_likelihood.postorder_partials[-1] * tf_scan_likelihood.preorder_partials[-1], axis=1)
tf.reduce_sum(tf_scan_likelihood.postorder_partials[-1, :, 0] * tf_scan_likelihood.pattern_counts / site_likelihoods)

<tf.Tensor: id=23281, shape=(), dtype=float64, numpy=25.16083296428147>

In [192]:
pi_diff = substitution_model.hky_frequencies_differential(kappa)[0]
pi_diff_norm = substitution_model.normalised_differential(pi_diff, hky_q_norm, hky_norm_const, pi)
freq_transition_prob_differentials = substitution_model.transition_probs_differential(pi_diff_norm, eigendecomp, branch_lengths)
tf.reduce_sum(tf_scan_likelihood.compute_edge_derivatives(freq_transition_prob_differentials))

<tf.Tensor: id=23155, shape=(), dtype=float64, numpy=3.640960580759664>